In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

In [3]:
# Create m x n matrix for Y
Y_train = np.loadtxt('./data/train_cleaned.txt').astype(int)
Y_test = np.loadtxt('./data/test_cleaned.txt').astype(int)

### Method 3 (Using scikit-surprise)

In [5]:
reader = Reader(rating_scale=(1, 5))
np.concatenate((Y_train, Y_test))
data = Dataset.load_from_df(pd.DataFrame(np.concatenate((Y_train, Y_test))), reader)

for reg in [10**-4, 10**-3, 0.01, 0.1, 1, 0]:
    print("reg: {}".format(reg))
    algo = SVD(n_factors = 20, biased=True, reg_all = reg, n_epochs = 40)
    trainset, testset = train_test_split(data, test_size=0.1)
    algo.fit(trainset)
    # User factors (u)
    u = algo.pu
    # Item factors (v)
    v = algo.qi
    predictions = algo.test(testset)
    print(accuracy.rmse(predictions))
print("\n\n")

for lr in [0.3, 0.03, 0.003,0.0003]:
    print("lr: {}".format(lr))
    algo = SVD(n_factors = 20, biased=True, reg_all = 0.1, lr_all = lr, n_epochs = 40)
    trainset, testset = train_test_split(data, test_size=0.1)
    algo.fit(trainset)
    # User factors (u)
    u = algo.pu
    # Item factors (v)
    v = algo.qi
    predictions = algo.test(testset)
    print(accuracy.rmse(predictions))

reg: 0.0001
RMSE: 0.9697
0.9696542703316504
reg: 0.001
RMSE: 0.9605
0.9604780502705889
reg: 0.01
RMSE: 0.9569
0.9569094664724928
reg: 0.1
RMSE: 0.9202
0.9201546304509504
reg: 1
RMSE: 0.9826
0.9825518901805712
reg: 0
RMSE: 0.9606
0.9605585878179745


lr: 0.3
RMSE: 1.8585
1.8584940139801367
lr: 0.03
RMSE: 0.9170
0.9170476306133247
lr: 0.003
RMSE: 0.9242
0.9242311446483035
lr: 0.0003
RMSE: 0.9747
0.9747412966532608


In [18]:
# Best setup
algo = SVD(n_factors = 20, biased=True, reg_all = 0.1, lr_all = 0.03, n_epochs = 40)
trainset, testset = train_test_split(data, test_size=0.1)
algo.fit(trainset)
# User factors (u)
U = algo.pu
# Item factors (v)
V = algo.qi
predictions = algo.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.9246
0.9245541427481175


## Visualize and Interpret Results

In [ ]:
# Visualize and interpret results
def visualize(U, V):
    ''' 
        U (k x M)
        V (k x N)
    '''
    A, sigma, B = np.linalg.svd(V)
    U_tilde = np.matmul(np.transpose(A[:,:2]), U)
    V_tilde = np.matmul(np.transpose(A[:,:2]), V)
    return U_tilde, V_tilde
    
U_tilde, V_tilde = visualize(np.transpose(U), np.transpose(V))

In [ ]:
colnames = ['movie_id', 'title', 'unknown', "action", "adventure", "animation", 
            "childrens", "comedy", "crime", "documentary", "drama", "fantasy", 
            "film-noir", "horror", "musical", "mystery", "romance", "sci-fi", 
            "thriller", "war", "western"]
movie_info = pd.read_csv("data/movies.txt", names=colnames, sep='\t', header=None)

movie_to_genre = dict()

for index, row in movie_info.iterrows():
    if row['title'] not in movie_to_genre:
        movie_to_genre[row['title']] = row['unknown':'western'].to_numpy()

id_to_movie_title = movie_info['title'].tolist()
id_to_movie_title.insert(0, "NO MOVIE WITH ID 0")

movie_to_id = dict()

for i in range(1,len(id_to_movie_title)):
    if (id_to_movie_title[i] not in movie_to_id):
        movie_to_id[id_to_movie_title[i]] = i        

### Plot 10 Movies from Entire Dataset

In [ ]:
np.random.seed(42)
movies = np.random.choice(list(movie_to_genre.keys()), 10, replace=False)
movie_ids = [movie_to_id[m] for m in movies]

# ====================== TEN CHOSEN MOVIES PLOT ======================
V_ten = np.take(np.transpose(V_tilde), movie_ids, axis=0)

fig = plt.figure(figsize=(11, 9), dpi=80)
ax = fig.add_subplot(1, 1, 1)

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')

# Eliminate upper and right axes
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Show ticks in the left and lower axes only
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

for i in range(len(V_ten)):
    x = V_ten[i][0]
    y = V_ten[i][1]
    plt.scatter(x, y, marker='.', color='blue', s=100)
    plt.text(x - 0.1, y + 0.01, movies[i], fontsize=11)
plt.title('Method 1: 10 Movies from MovieLens dataset', fontsize=16, fontweight='bold')
plt.show()

### Plot 10 Most Popular Movies

In [ ]:
popular_movies = ['Star Wars (1977)', 'Contact (1997)', 'Fargo (1996)', 
                  'Return of the Jedi (1983)', 'Liar Liar (1997)', 
                  'English Patient, The (1996)', 'Scream (1996)', 
                  'Toy Story (1995)', 'Air Force One (1997)', 
                  'Independence Day (ID4) (1996)']
pop_movies_id = [movie_to_id[m] for m in popular_movies]

# ====================== TEN MOST POPULAR MOVIES PLOT ======================
V_pop = np.take(np.transpose(V_tilde), pop_movies_id, axis=0)

fig = plt.figure(figsize=(11, 9), dpi=80)
ax = fig.add_subplot(1, 1, 1)

# Center axes
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

for i in range(len(V_pop)):
    x = V_pop[i][0]
    y = V_pop[i][1]
    plt.scatter(x, y, marker='.', color='blue', s=100)
    plt.text(x - 0.1, y + 0.01, popular_movies[i], fontsize=11)
plt.title('Method 1: 10 Most Popular Movies', fontsize=16, fontweight='bold')
plt.show()

### Plot 10 Best Movies

In [ ]:
best_movies = ['Prefontaine (1997)', 'Santa with Muscles (1996)', 
               'Great Day in Harlem, A (1994)', 'Aiqing wansui (1994)', 
               'Star Kid (1997)', "Someone Else's America (1995)", 
               'Entertaining Angels: The Dorothy Day Story (1996)', 
               'Saint of Fort Washington, The (1993)', 
               'They Made Me a Criminal (1939)', 
               'Marlene Dietrich: Shadow and Light (1996) ']

best_movies_id = [movie_to_id[m] for m in best_movies]

# ====================== TEN BEST MOVIES PLOT ======================
V_best = np.take(np.transpose(V_tilde), best_movies_id, axis=0)

fig = plt.figure(figsize=(11, 9), dpi=80)
ax = fig.add_subplot(1, 1, 1)

# Center axes
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

for i in range(len(V_best)):
    x = V_best[i][0]
    y = V_best[i][1]
    plt.scatter(x, y, marker='.', color='blue', s=100)
    plt.text(x - 0.1, y + 0.01, best_movies[i], fontsize=11)
plt.title('Method 1: 10 Best Movies', fontsize=16, fontweight='bold')
plt.show()